
# Maze Pathfinding Benchmark — Shared Grid, Reproducible

This notebook benchmarks multiple algorithms on the **same grid** for fairness.  
It runs on 21×21 and 31×31 mazes, with two map types:
- **dense**: perfect maze (tree, no loops)
- **sparse**: ~5% walls removed to create loops

Metrics captured per algorithm: **runtime (ms)**, **nodes expanded**, **path length** (nodes and edges), and **success**.  
Outputs: CSV and charts saved to `./artifacts/`.


In [ ]:

# 0) Environment and configuration
from __future__ import annotations
import time, math, csv, os, heapq, random
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Callable, Optional

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import deque
from pathlib import Path

# Artifact directory
ARTIFACTS_DIR = Path("./artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True, parents=True)

# For deterministic plots
np.set_printoptions(linewidth=120, threshold=20)


## 1) Maze generation (deterministic DFS)

In [ ]:

def generate_maze_dfs_seeded(width:int, height:int, seed:int=0)->np.ndarray:
    """DFS backtracker with seeded randomness. Walls=1, paths=0."""
    rnd = random.Random(seed)
    maze = np.ones((height, width), dtype=int)
    stack = [(1,1)]
    maze[1,1] = 0
    dirs = [(-2,0),(2,0),(0,-2),(0,2)]
    while stack:
        y,x = stack.pop()
        rnd.shuffle(dirs)
        for dy,dx in dirs:
            ny,nx = y+dy, x+dx
            if 1<=ny<height-1 and 1<=nx<width-1 and maze[ny,nx]==1:
                maze[ny,nx] = 0
                maze[y+dy//2, x+dx//2] = 0
                stack.append((ny,nx))
    # ensure start/end open
    maze[1,1] = 0
    maze[height-2, width-2] = 0
    return maze

def add_loops(maze: np.ndarray, p: float, seed:int=0)->np.ndarray:
    """Make the grid more open by randomly knocking out walls with prob p."""
    rnd = np.random.default_rng(seed)
    m = maze.copy()
    h,w = m.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if m[y,x] == 1 and rnd.random() < p:
                m[y,x] = 0
    return m


## 2) Solvers with tracing (DFS, BFS, Dijkstra, A*)

In [ ]:

def dfs_trace(maze: np.ndarray, start: Tuple[int,int], end: Tuple[int,int]) -> Dict[str, Any]:
    h, w = maze.shape
    visited = np.zeros((h,w), dtype=bool)
    stack = [(start[0], start[1], [start])]
    visited[start] = True
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]
    expanded = 0
    while stack:
        y,x,path = stack.pop()
        expanded += 1
        if (y,x) == end:
            return {"path": path, "nodes_expanded": expanded}
        for dy,dx in dirs:
            ny,nx = y+dy, x+dx
            if 0<=ny<h and 0<=nx<w and maze[ny,nx]==0 and not visited[ny,nx]:
                visited[ny,nx] = True
                stack.append((ny,nx, path+[(ny,nx)]))
    return {"path": [], "nodes_expanded": expanded}

def bfs_trace(maze: np.ndarray, start: Tuple[int,int], end: Tuple[int,int]) -> Dict[str, Any]:
    h, w = maze.shape
    visited = np.zeros((h,w), dtype=bool)
    parent: Dict[Tuple[int,int], Optional[Tuple[int,int]]] = {start: None}
    q = deque([start])
    visited[start] = True
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]
    expanded = 0
    while q:
        y,x = q.popleft()
        expanded += 1
        if (y,x) == end:
            path = []
            cur = end
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path.reverse()
            return {"path": path, "nodes_expanded": expanded}
        for dy,dx in dirs:
            ny,nx = y+dy, x+dx
            if 0<=ny<h and 0<=nx<w and maze[ny,nx]==0 and not visited[ny,nx]:
                visited[ny,nx] = True
                parent[(ny,nx)] = (y,x)
                q.append((ny,nx))
    return {"path": [], "nodes_expanded": expanded}

def dijkstra_trace(maze: np.ndarray, start: Tuple[int,int], end: Tuple[int,int]) -> Dict[str, Any]:
    h, w = maze.shape
    dist = np.full((h,w), np.inf)
    dist[start] = 0.0
    parent: Dict[Tuple[int,int], Optional[Tuple[int,int]]] = {start: None}
    pq: List[Tuple[float, Tuple[int,int]]] = [(0.0, start)]
    closed = set()
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]
    expanded = 0
    while pq:
        d, cur = heapq.heappop(pq)
        if cur in closed:
            continue
        closed.add(cur)
        expanded += 1
        if cur == end:
            path = []
            x = cur
            while x is not None:
                path.append(x); x = parent[x]
            path.reverse()
            return {"path": path, "nodes_expanded": expanded}
        cy,cx = cur
        for dy,dx in dirs:
            ny,nx = cy+dy, cx+dx
            if 0<=ny<h and 0<=nx<w and maze[ny,nx]==0:
                cand = d + 1.0
                if cand < dist[ny,nx]:
                    dist[ny,nx] = cand
                    parent[(ny,nx)] = cur
                    heapq.heappush(pq, (cand, (ny,nx)))
    return {"path": [], "nodes_expanded": expanded}

def astar_trace(maze: np.ndarray, start: Tuple[int,int], end: Tuple[int,int]) -> Dict[str, Any]:
    h, w = maze.shape
    def hfun(p: Tuple[int,int]) -> int:
        return abs(p[0]-end[0]) + abs(p[1]-end[1])
    g = {start: 0}
    parent: Dict[Tuple[int,int], Optional[Tuple[int,int]]] = {start: None}
    pq: List[Tuple[int,int,Tuple[int,int]]] = [(hfun(start), 0, start)]
    closed = set()
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]
    expanded = 0
    while pq:
        f, gc, cur = heapq.heappop(pq)
        if cur in closed:
            continue
        closed.add(cur)
        expanded += 1
        if cur == end:
            path = []
            x = cur
            while x is not None:
                path.append(x); x = parent[x]
            path.reverse()
            return {"path": path, "nodes_expanded": expanded}
        cy,cx = cur
        for dy,dx in dirs:
            ny,nx = cy+dy, cx+dx
            if 0<=ny<h and 0<=nx<w and maze[ny,nx]==0:
                cand = gc + 1
                if cand < g.get((ny,nx), 10**9):
                    g[(ny,nx)] = cand
                    parent[(ny,nx)] = cur
                    heapq.heappush(pq, (cand + hfun((ny,nx)), cand, (ny,nx)))
    return {"path": [], "nodes_expanded": expanded}


## 3) Benchmark harness and plotting

In [ ]:

@dataclass
class RunResult:
    algo: str
    size: str        # e.g., "21x21"
    map_type: str    # "dense" or "sparse"
    success: int
    path_len: int    # nodes in path
    edges: int       # path_len-1 if success else 0
    expanded: int
    runtime_ms: float

def timed(solver: Callable[[np.ndarray, Tuple[int,int], Tuple[int,int]], Dict[str,Any]], 
          maze: np.ndarray,
          start: Tuple[int,int],
          end: Tuple[int,int]) -> Dict[str,Any]:
    t0 = time.perf_counter()
    out = solver(maze, start, end)
    t1 = time.perf_counter()
    out["runtime_ms"] = (t1 - t0) * 1e3
    return out

def benchmark_once(maze: np.ndarray, name: str, solver, size_label: str, map_type: str, start: Tuple[int,int], end: Tuple[int,int]) -> RunResult:
    out = timed(solver, maze, start, end)
    path = out.get("path", [])
    success = int(len(path) > 0 and path[-1] == end)
    L = len(path)
    edges = max(0, L-1) if success else 0
    return RunResult(
        algo=name, size=size_label, map_type=map_type, success=success,
        path_len=L, edges=edges, expanded=out.get("nodes_expanded", 0),
        runtime_ms=out.get("runtime_ms", float("nan"))
    )

def plot_metric_bars(rows: List[RunResult], metric: str, fname: str, title: str):
    conditions = sorted({(r.size, r.map_type) for r in rows})
    for size, mtype in conditions:
        subset = [r for r in rows if r.size==size and r.map_type==mtype]
        if not subset:
            continue
        names = [r.algo for r in subset]
        vals = [getattr(r, metric) for r in subset]
        plt.figure(figsize=(6,4))
        plt.bar(names, vals)
        plt.ylabel(metric.replace("_", " "))
        plt.title(f"{title} — {size}, {mtype}")
        for i, v in enumerate(vals):
            label = f"{v:.1f}" if isinstance(v, float) else f"{v}"
            plt.text(i, v, label, ha="center", va="bottom", fontsize=9)
        plt.tight_layout()
        outpath = ARTIFACTS_DIR / f"{fname}_{size}_{mtype}.png"
        plt.savefig(outpath, dpi=160)
        plt.show()


## 4) Run experiments on shared grids

In [ ]:

seed = 42
sizes = [(21,21), (31,31)]
map_types = {
    "dense": 0.00,   # original perfect maze
    "sparse": 0.05,  # 5% random wall removals to add loops/open areas
}
algs: List[Tuple[str, Callable]] = [
    ("DFS", dfs_trace),
    ("BFS", bfs_trace),
    ("Dijkstra", dijkstra_trace),
    ("A*", astar_trace),
]

rows: List[RunResult] = []
for (H,W) in sizes:
    H |= 1; W |= 1  # enforce odd
    base = generate_maze_dfs_seeded(W, H, seed=seed)
    start, end = (1,1), (H-2, W-2)
    for mtype, p in map_types.items():
        grid = base if p == 0.0 else add_loops(base, p=p, seed=seed)
        grid.setflags(write=False)
        size_label = f"{H}x{W}"
        for name, fn in algs:
            res = benchmark_once(grid, name, fn, size_label, mtype, start, end)
            rows.append(res)

print(f"Total runs: {len(rows)}")            
for r in rows:
    print(f"{r.size:8} {r.map_type:6} {r.algo:9} | success={r.success} edges={r.edges} expanded={r.expanded} time={r.runtime_ms:.2f} ms")


## 5) Save CSV of results

In [ ]:

csv_path = ARTIFACTS_DIR / "maze_benchmarks.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["algo","size","map_type","success","path_len","edges","expanded","runtime_ms"])
    for r in rows:
        w.writerow([r.algo, r.size, r.map_type, r.success, r.path_len, r.edges, r.expanded, round(r.runtime_ms,3)])
csv_path


## 6) Figures

In [ ]:

plot_metric_bars(rows, "runtime_ms", "runtime_ms", "Runtime (ms) by algorithm")
plot_metric_bars(rows, "expanded",   "expanded",   "Nodes expanded by algorithm")
plot_metric_bars(rows, "edges",      "edges",      "Path length (edges) by algorithm")


## 7) Preview table

In [ ]:

import pandas as pd
df = pd.DataFrame([r.__dict__ for r in rows]).sort_values(["size","map_type","algo"])
df
